In [1]:
import sys
sys.path.append('scripts/dialent/task1/')
sys.path.append('scripts/dialent/')
sys.path.append('scripts/')
import util

In [2]:
import pandas as pd
import random

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb

In [4]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [5]:
def tkn_dict(u):
    d = dict()
    for i in range(len(u)):
        tokens = u[i].tokens
        for j in tokens:
            d[int(j.id)] = j.text
    return d

In [6]:
def ne_bin(x):
    if x == 'LOC':
        return 0
    elif x == 'LOCORG':
        return 1
    return 2

In [7]:
def get_df(u):
    df = pd.DataFrame()
    for i in range(len(u)):
        res = u[i].makeTokenSets()
        for j in range(len(res)):
            token = res[j].toInlineString()
            split = token.split()
            token_id = int(str(res[j]).split('#')[-1][:-2])
            df = df.append({'tokens': split[4:], 
                            'tokens_id': token_id,
                            'obj_id': int(split[1]),
                            'ne_bin': ne_bin(str(split[0]))}, ignore_index=True)
    return df

In [8]:
CTX = 3
def get_from_dict_dev(id_d):
    d = tkn_dev_dict
    ans = str()
    for i in range(-CTX, CTX):
        if id_d + i in d:
            ans += " " + str(d[id_d + i])
    return ans

def get_from_dict_test(id_d):
    d = tkn_test_dict
    ans = str()
    for i in range(-CTX, CTX):
        if id_d + i in d:
            ans += " " + str(d[id_d + i])
    return ans

In [9]:
u_dev = util.loadAllStandard('./devset/')
u_test = util.loadAllStandard('./testset/')

Failed to load the standard of book_3954:
Unknown mention tag: Facility


In [10]:
tkn_dev_dict = tkn_dict(u_dev)
tkn_test_dict = tkn_dict(u_test)

In [11]:
df_dev = get_df(u_dev)
df_dev['ctx'] = df_dev['tokens_id'].apply(get_from_dict_dev)
df_dev = df_dev[df_dev['ne_bin'] < 2]

In [12]:
df_test = get_df(u_test)
df_test['ctx'] = df_test['tokens_id'].apply(get_from_dict_test)
df_test = df_test[df_test['ne_bin'] < 2]

In [13]:
n = df_dev.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
                      min_df=3, max_df=0.9, strip_accents='unicode',
                      use_idf=1, smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(df_dev['ctx'])
test_term_doc = vec.transform(df_test['ctx']) 

In [14]:
Y_dev = df_dev['ne_bin']
Y_test = df_test['ne_bin']

In [15]:
best_parameters = {
    'learning_rate': 0.08,
    'max_depth': 5
}

In [16]:
clf = xgb.XGBClassifier(**best_parameters)
clf.fit(trn_term_doc, Y_dev)
predict = clf.predict(test_term_doc)

u = util.loadAllStandard('./testset/')
for i in range(len(u)):
    res = u[i].makeTokenSets()
    row = []
    for j in range(len(res)):
        token = res[j].toInlineString()
        if token[0] == 'L':
            split = token.split()
            if predict[df_test['obj_id'] == int(split[1])][0] == 0:
                tag = 'LOC'
            else:
                tag = 'LOCORG'
            start = int(split[2][1:-1])
            end = int(split[3][:-1])
            row.append('%s %d %d\n' % (tag, start, end-start+1))
            #if tag != split[0]:
            #    print(token)
        else:
            split = token.split()
            start = int(split[2][1:-1])
            end = int(split[3][:-1])
            row.append('%s %d %d\n' % (split[0], start, end-start+1))
    with open('./result/' + u[i].name + '.task1', 'w') as f:
        f.writelines(row)   

!python scripts/t1_eval.py -s ./testset -t ./result

Failed to load the standard of book_3954:
Unknown mention tag: Facility
Failed to load the standard of book_3954:
Unknown mention tag: Facility
Type    P        R        F1       TP1      TP2      In Std.  In Test.
per        0.9993   0.9993   0.9993  1342.00  1342.00     1343     1343
loc        0.7285   0.6969   0.7123   417.43   417.43      599      573
org        0.9895   0.9895   0.9895  1557.55  1557.55     1574     1574
locorg     0.6686   0.8070   0.7313   510.83   510.83      633      764
overall    0.8996   0.9224   0.9109  3819.81  3819.81     4141     4246
